参考  
[言語処理100本ノック 2020 第10章: 機械翻訳 (90-98)](https://qiita.com/nymwa/items/867e05a43060d036a174)  
@nymwa

# 90. データの準備

In [9]:
#!cat ./data/kftt-data-1.0/data/orig/kyoto-train.ja | sed 's/\s+/ /g' | ginzame > ./data/train.ginza.ja
#!cat ./data/kftt-data-1.0/data/orig/kyoto-dev.ja | sed 's/\s+/ /g' | ginzame > ./data/dev.ginza.ja
#!cat ./data/kftt-data-1.0/data/orig/kyoto-test.ja | sed 's/\s+/ /g' | ginzame > ./data/test.ginza.ja

In [12]:
"""
for src, dst in [
    ('./data/train.ginza.ja', './data/train.spacy.ja'),
    ('./data/dev.ginza.ja', './data/dev.spacy.ja'),
    ('./data/test.ginza.ja', './data/test.spacy.ja'),
]:
    with open(src) as f:
        lst = []
        tmp = []
        for x in f:
            x = x.strip()
            if x == 'EOS':
                lst.append(' '.join(tmp))
                tmp = []
            elif x != '':
                tmp.append(x.split('\t')[0])
    with open(dst, 'w') as f:
        for line in lst:
            print(line, file=f)
"""

In [21]:
with open('./data/train.spacy.ja') as f:
    data = f.readlines()
data[0]

'雪舟 （ せっ しゅう 、 1420 年 （ 応永 27 年 ） - 1506 年 （ 永正 3 年 ） ） は 号 で 、 15 世紀 後半 室町 時代 に 活躍 し た 水墨 画家 ・ 禅僧 で 、 画聖 と も 称え られる 。\n'

import re
import spacy

nlp = spacy.load('en')
for src, dst in [
    ('./data/kftt-data-1.0/data/orig/kyoto-train.en', './data/train.spacy.en'),
    ('./data/kftt-data-1.0/data/orig/kyoto-dev.en', './data/dev.spacy.en'),
    ('./data/kftt-data-1.0/data/orig/kyoto-test.en', './data/test.spacy.en'),
]:
    with open(src) as f, open(dst, 'w') as g:
        for x in f:
            x = x.strip()
            x = re.sub(r'\s+', ' ', x)
            x = nlp.make_doc(x)
            x = ' '.join([doc.text for doc in x])
            print(x, file=g)

In [22]:
with open('./data/train.spacy.en') as f:
    data = f.readlines()
data[0]

'Known as Sesshu ( 1420 - 1506 ) , he was an ink painter and Zen monk active in the Muromachi period in the latter half of the 15th century , and was called a master painter .\n'

# 91. 機械翻訳モデルの訓練

In [27]:
!fairseq-preprocess -s ja -t en \
    --trainpref data/train.spacy \
    --validpref data/dev.spacy \
    --destdir data91  \
    --thresholdsrc 5 \
    --thresholdtgt 5 \
    --workers 20

Namespace(align_suffix=None, alignfile=None, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data91', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer='nag', padding_factor=8, seed=1, source_lang='ja', srcdict=None, target_lang='en', task='translation', tensorboard_logdir='', testpref=None, tgtdict=None, threshold_loss_scale=None, thresholdsrc=5, thresholdtgt=5, tokenizer=None, trainpref='data/train.spacy', user_dir=None, validpref='data/dev.spacy', workers=20)
| [ja] Dictionary: 60247 types
| [ja] data/train.spacy.ja: 440288 sents, 11298955 tokens, 1.41% replaced by <unk>
| [ja] Dictionary: 60247 types
| [ja] data/dev.spacy.ja: 1166 sents, 25550 tokens, 1.54% replaced by <unk>
| [en] Diction

In [29]:
!fairseq-train data91 \
    --save-dir save91 \
    --max-epoch 10 \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --clip-norm 1.0 \
    --lr 1e-3 --lr-scheduler inverse_sqrt --warmup-updates 2000 \
    --update-freq 1 \
    --dropout 0.2 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 8000 > 91.log

| epoch 001:   3%| | 61/1807 [1:43:27<46:35:54, 96.08s/it, loss=14.405, nll_loss^C
